# 第7章: データベース
[artist.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz)はオープンな音楽データベース[MusicBrainz](https://musicbrainz.org/)の中でアーティストに関するものをJSON形式に変換し、gzip形式で圧縮したファイルである。このファイルには１アーティストに関する情報が1行にJSON形式で格納されている。JSON形式は以下の通りである。

|フィールド|     型     | 内容|例|
|--------|------------|-|-|
|   id   |ユニーク識別子|整数|20660|
|gid	|グローバル識別子	|文字列	|"ecf9f3a3-35e9-4c58-acaa-e707fba45060"|
|name	|アーティスト名	|文字列	|"Oasis"|
|sort_name	|アーティスト名（辞書順整列用）	|文字列	|"Oasis"|
|area	|活動場所	|文字列	|"United Kingdom"|
|aliases	|別名|	辞書オブジェクトのリスト	||
|aliases[].name|別名	|文字列	|"オアシス"|
|aliases[].sort_name	|別名（整列用）	|文字列|	"オアシス"|
|begin	|活動開始日	|辞書	|
|begin.year|	活動開始年|	整数|	1991|
|begin.month	|活動開始月|	整数	|
|begin.date	|活動開始日|	整数	|
|end|	活動終了日|	辞書	|
|end.year|	活動終了年|	整数|	2009|
|end.month|	活動終了月|	整数|	8|
|end.date|	活動終了日|	整数|	28|
|tags|	タグ|	辞書オブジェクトのリスト|	|
|tags[].count|	タグ付けされた回数|	整数|	1|
|tags[].value|	タグ内容|	文字列|	"rock"|
|rating|	レーティング|	辞書オブジェクト	|
|rating.count|	レーティングの投票数|	整数|	13|
|rating.value|	レーティングの値（平均値）|	整数|	86|

artist.json.gzのデータをKey_Value_Store(KVS)およびドキュメント志向型データベースに格納・検索することを考える。KVSとしては、LebelDB, Redis, KyotoCabinet等を用いよ。ドキュメント志向型データベースとしては、MongoDBを採用したが、CouchDBやRethinkDB等を用いても良い。

### 60. KVSの構築
Key-Value-Store(KVS)を用い、アーティスト名(name)から活動場所(area)を検索するためのデータベースを構築せよ。

In [5]:
import redis
import json
file = open('./resource/artist.json','r')
artist_jsonList = []
for line in file:
    artist_json = json.loads(line)
    artist_jsonList.append(artist_json)

In [7]:
r = redis.Redis(host='localhost', port=6379, db=0)
for json in artist_jsonList:
    if 'name'in json and 'area' in json:
        r.set(json['name'], json['area'])  

### 61. KVSの検索
60で構築したデータベースを用い、特定の(指定された)アーティストの活動場所を取得せよ。

In [8]:
r.get("Oasis").decode()

'United Kingdom'

### 62. KVS内の反復処理
60で構築したデータベースを用い、活動場所がJapanとなっているアーティスト数を求めよ。

In [9]:
count = 0
for i in r.scan_iter():
    if r.get(i) == b"Japan":
        count += 1
count

22128

### 63. オブジェクトを値に格納したKVS
KVSを用い、アーティスト名(name)からタグと被タグ数(タグづけされた回数)のリストを検索するためのデータベースを構築せよ。さらにここで構築したデータベースを用い、アーティスト名からタグと被タグ数を検索せよ。

In [11]:
tag_redis = redis.Redis(host='localhost', port=6379, db=1)
for json in artist_jsonList:
    if 'tags' in json:
        for tag in json['tags']:
            tag_redis.hset(json['name'], tag['value'], tag['count'])

In [18]:
tag_redis.hgetall('Infester')

{b'american': b'1', b'death metal': b'1'}

### 64.MongoDBの構築
アーティスト情報(artist.json.gz)をデータベースに登録せよ。さらに、次のフィールドでインデックスを作成せよ:name,aliases.name, tags.value, rating.value
```bash
sudo mongod --dbpath ./python/100本ノック/chapter07/resource/Mongo --logpath ./python/100本ノック/chapter07/resource/mongodb.log
```

In [15]:
from pymongo import MongoClient,ASCENDING,DESCENDING

In [7]:
from pymongo import MongoClient,ASCENDING,DESCENDING
client = MongoClient('localhost', 27017)
db = client.artist
collection = db.artist
for x in range(0,len(artist_jsonList),10000):
    if len(artist_jsonList) - x >10000:
        temp = artist_jsonList[x:10000+x]
        collection.insert_many(temp)
    else:
        temp = artist_jsonList[x:len(artist_jsonList)]
        collection.insert_many(artist_jsonList[x:len(artist_jsonList)])
#         collection.insert_many(artist_jsonList[x:10000+x])
#     print("END:{} times".format(x))

collection.create_index([('name', ASCENDING)])  
collection.create_index([('aliases.name', ASCENDING)])  
collection.create_index([('tags.value', ASCENDING)])
collection.create_index([('rating.value', ASCENDING)])

'rating.value_1'

```Bash
show databases
use artist
db.artist.find({'name':'Queen'})
```

In [8]:
for data in collection.find({'name': 'Queen'}):
    print (data)

{'_id': ObjectId('5b9e0376e2010e68c0ca9dc3'), 'name': 'Queen', 'area': 'Japan', 'gender': 'Female', 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'sort_name': 'Queen', 'ended': True, 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', 'type': 'Character', 'id': 701492, 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}
{'_id': ObjectId('5b9e0377e2010e68c0cb646f'), 'rating': {'count': 24, 'value': 92}, 'begin': {'date': 27, 'month': 6, 'year': 1970}, 'name': 'Queen', 'area': 'United Kingdom', 'tags': [{'count': 2, 'value': 'hard rock'}, {'count': 1, 'value': '70s'}, {'count': 1, 'value': 'queen family'}, {'count': 1, 'value': '90s'}, {'count': 1, 'value': '80s'}, {'count': 1, 'value': 'glam rock'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'english'}, {'count': 2, 'value': 'uk'}, {'count': 1, 'value': 'pop/rock'}, {'count': 1, 'value': 'pop-rock'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'classic pop and rock'},

### 66.検索件数の取得
MongoDBのインタラクティブシェルを用いて活動場所が「Japan」となっているアーティスト数を求めよ。  
メモ:$gt→Greater Than
```Bash:JapaneseArtistCount
db.artist.find({"area":"Japan"}).count()
22821
```

### 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．
```Bash
db.artist.find({"aliases":{ $elemMatch :{"name":"Queen"}}})
```

In [14]:
for data in collection.find({"aliases.name":"Queen"}):
    print(data)

{'_id': ObjectId('5b6bd146e2010e02dab4da9f'), 'name': 'Queen', 'area': 'Japan', 'gender': 'Female', 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'sort_name': 'Queen', 'ended': True, 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', 'type': 'Character', 'id': 701492, 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}


### 68.ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティストトップ10を求めよ

In [20]:
result = collection.find({"tags.value":"dance"})
for data in result.sort('rating.count', DESCENDING)[0:10]:
    print(data['name'], data['rating'])

Madonna {'count': 26, 'value': 88}
Björk {'count': 23, 'value': 84}
The Prodigy {'count': 23, 'value': 90}
Rihanna {'count': 15, 'value': 68}
Britney Spears {'count': 13, 'value': 83}
Maroon 5 {'count': 11, 'value': 60}
Adam Lambert {'count': 7, 'value': 100}
Fatboy Slim {'count': 7, 'value': 77}
Basement Jaxx {'count': 6, 'value': 83}
Cornershop {'count': 5, 'value': 68}
